## torch + transformers で自作モデルを作る際の基礎の基礎

In [1]:
import os
os.makedirs('./storage', exist_ok=True)

MODEL_NAME = 'starter-model'

#### Config の定義
---
* `PretrainedConfig` を継承する形で定義
* 利点としては以下
    * 既に定義された変数が使えたり，セーブ・ロードがお手軽にできたりする
    * `transformers` の既存 `Config` とインターフェイスが合う

In [2]:
from transformers import PretrainedConfig
class Config(PretrainedConfig):
    def __init__(
        self,
        ## __init__ 関数では必ずデフォルト引数を設定すること
        input_size: int = 5,
        hidden_size: int = 3,
        output_size: int = 1,
        ## 継承元のPretrainedConfig クラスで既に実装されているパラメータはここで指定
        ## 親クラスのコンストラクタ super()の呼び出し時に渡す
        **kwards
    ):
        ## kwards を親クラスのインスタンスに渡す
        ## PretrainedConfig クラスの変数をオーバーライド
        super().__init__(**kwards)

        ## モデルの独自変数に関してはここで、クラスのメンバ変数として渡す
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

/Users/tomoyahirakawa/.pyenv/versions/3.11.6/envs/dnn-dev/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
## デフォルト引数が定義されているため、
## 何も引数がない場合でも宣言可能
config = Config()
print(config)

Config {
  "hidden_size": 3,
  "input_size": 5,
  "output_size": 1,
  "transformers_version": "4.38.2"
}



In [4]:
## 普通は、モデルの変数を指定するはず
config = Config(
    input_size=999,
    hidden_size=1000,
    output_size=999
)
print(config)

Config {
  "hidden_size": 1000,
  "input_size": 999,
  "output_size": 999,
  "transformers_version": "4.38.2"
}



In [5]:
## 保存と読み込み

config = Config()
outdir = os.path.join('storage', MODEL_NAME)
config.save_pretrained(outdir)
loaded_config = Config.from_pretrained(outdir)
print(config)
print(loaded_config)

Config {
  "hidden_size": 3,
  "input_size": 5,
  "output_size": 1,
  "transformers_version": "4.38.2"
}

Config {
  "hidden_size": 3,
  "input_size": 5,
  "output_size": 1,
  "transformers_version": "4.38.2"
}



#### Model の定義
---
* `PretrainedModel` を継承する形で定義
* 以下注意点
    * クラス変数として `config_class` に `Config` を定義すること(`from_pretrained()`でモデルを読み込む際に必ず必要になる）。
    * `def __init__(self, config):` のように、初期化時に `Config` のインスタンスを渡すこと。
    * `super().__init__(config)`のように、親クラスの初期化のためにも `Config` のインスタンスを渡すこと。

In [6]:
from transformers import PreTrainedModel
import torch.nn as nn

## 例として、単純な二層線形 NN
class Model(PreTrainedModel):
    config_class = Config
    def __init__(self, config):
        super().__init__(config)
        self.config = config
        self.linear1 = nn.Linear(
            self.config.input_size,
            self.config.hidden_size
        )
        self.linear2 = nn.Linear(
            self.config.hidden_size,
            self.config.output_size
        )

In [7]:
## こんな感じでで Model のインスタンスを作る
config = Config()
model  = Model(config)
print(model)

Model(
  (linear1): Linear(in_features=5, out_features=3, bias=True)
  (linear2): Linear(in_features=3, out_features=1, bias=True)
)


In [8]:
## Config 通りにモデルができる。
config = Config(
    input_size=999,
    hidden_size=1000,
    output_size=999
)
model  = Model(config)
print(model)

Model(
  (linear1): Linear(in_features=999, out_features=1000, bias=True)
  (linear2): Linear(in_features=1000, out_features=999, bias=True)
)


In [9]:
#### 保存と読み込み

config = Config()
model  = Model(config)
outdir = os.path.join('storage', MODEL_NAME)
model.save_pretrained(outdir)
loaded_model = Model.from_pretrained(outdir)

print(model)
print(loaded_model)

Model(
  (linear1): Linear(in_features=5, out_features=3, bias=True)
  (linear2): Linear(in_features=3, out_features=1, bias=True)
)
Model(
  (linear1): Linear(in_features=5, out_features=3, bias=True)
  (linear2): Linear(in_features=3, out_features=1, bias=True)
)


#### モデルの学習ループ
---
* 以下が Hugging Face っぽい実装みたい
    * `forward()` は、引数にラベルを渡すことができて，ラベルを渡された時にはlossまで計算して返すようにする
    * 返り値のために専用の`Output` クラス（`dataclass`）を別に用意し，返り値はそれに入れて返す


In [10]:
from transformers import PreTrainedModel
import torch
import torch.nn as nn
from dataclasses import dataclass
# 返り値用のクラス
@dataclass
class ModelOutput:
    loss: torch.Tensor = None
    logits: torch.Tensor = None

class Model(PreTrainedModel):
    config_class = Config
    def __init__(self, config):
        super().__init__(config)
        self.config = config
        self.linear1 = nn.Linear(
            self.config.input_size,
            self.config.hidden_size
        )
        self.linear2 = nn.Linear(
            self.config.hidden_size,
            self.config.output_size
        )

    def forward(
        self,
        input,
        labels=None
    ):
        logits = self.linear1(input)
        logits = self.linear2(logits)
        loss = None
        if labels is not None: # labels=に何か与えられていたら損失計算
            loss_fn = torch.nn.MSELoss()
            loss = loss_fn(
                logits,
                labels
            )
        return ModelOutput(
            loss=loss,
            logits=logits
        )

In [11]:
config = Config()
model = Model(config)
inputs = torch.rand((5))  # 何かの入力
labels = torch.tensor([0.5])  # 何かのラベル
output = model(inputs, labels)
print(output.loss)  # tensor(0.9413, grad_fn=<MseLossBackward0>)
output.loss.backward()

tensor(0.1837, grad_fn=<MseLossBackward0>)
